In [7]:
import numpy as np
from gurobipy import *

## Precision LP

In [8]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Email, S, t= multidict({i: [S[i], i*100] for i in range(10)})

In [12]:
model = Model('precision')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
I = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')
u = model.addVar(lb=0.0, ub =1.0, vtype='C', name = 'Sum_Indication')

# Time field

Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-End')

In [13]:
# Weight-Sum constraints
model.addConstr(quicksum(w[i] for i in range(10)) == 1.0, 'Sum(w[i]) == 1.0')

for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*I[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-I[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    model.addGenConstrAnd(I[i], [Is[i], Ie[i]], "TimeCon")

    model.addConstr(ts <= te, 'start <= end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 1)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 1)

model.setObjective(w.prod(S), GRB.MAXIMIZE)

model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)

Optimize a model with 51 rows, 43 columns and 110 nonzeros
Model has 50 general constraints
Variable types: 11 continuous, 32 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 1e+04]
Presolve removed 31 rows and 10 columns
Presolve time: 0.00s
Presolved: 110 rows, 93 columns, 246 nonzeros
Presolved model has 36 SOS constraint(s)
Variable types: 47 continuous, 46 integer (44 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 0.5000000

Root relaxation: objective 1.000000e+00, 56 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    9    0.50000    1.00000   100%     -    0s
H    0     0                       0.5000005    1.00000   100%     -    0s
     0     2    1.00000    

## Recall LP

In [15]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
u = 1/sum(S)

Email, S, t= multidict({i: [S[i], i*100] for i in range(10)})

In [16]:
model = Model('recall')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
I = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')

# Time field

Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='C', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='C', name = 'time-End')

In [17]:
# Time constraints
for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*I[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-I[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    model.addGenConstrAnd(I[i], [Is[i], Ie[i]], "TimeCon")
    
    model.addConstr(ts <= te, 'start <= end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0.0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0.0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 0.01)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 0.01)

model.setObjective(w.prod(S), GRB.MAXIMIZE)

model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)

Optimize a model with 50 rows, 42 columns and 80 nonzeros
Model has 50 general constraints
Variable types: 12 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+02]
  RHS range        [2e-01, 1e+04]
Presolve removed 61 rows and 20 columns
Presolve time: 0.00s
Presolved: 79 rows, 82 columns, 166 nonzeros
Presolved model has 36 SOS constraint(s)
Variable types: 38 continuous, 44 integer (44 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 1.000000e+00, 39 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    7   -0.00000    1.00000      -     -    0s
     0     2    1.00000    0    7   -0.00000    1.00000      -     -    0s
*   40     9               8       0.4000000    1.00000   150%   1.1    0s


## F1-Score LP

In [18]:
S = [1,0,0,0,1,0,0,0,1,1]
#S = [0,0,0,1,1,0,0,1,1,1]   
C = sum(S)

#Email, S, t= multidict({i: [S[i], t[i]] for i in range(10)})
Email, S, t= multidict({i: [S[i], i*100] for i in range(10)})

print(t)

{0: 0, 1: 100, 2: 200, 3: 300, 4: 400, 5: 500, 6: 600, 7: 700, 8: 800, 9: 900}


In [19]:
model = Model('F1')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
I = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')
u = model.addVar(lb=0.01, ub =1.0, vtype='C', name = 'Sum_Indication')

# Time field
Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='C', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='C', name = 'time-End')

In [20]:
model.addConstr(quicksum(w[i] for i in range(10))+u*C == 1.0, 'sum(w)+uC=1')

for i in Email:
    
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*I[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-I[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    model.addGenConstrAnd(I[i], [Is[i], Ie[i]], "TimeCon")
    
    model.addConstr(ts <= te, 'start <= end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 1)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 1)

model.setObjective(w.prod(S), GRB.MAXIMIZE)

model.optimize()
                 
for v in model.getVars():
    print(v.varName, v.x)

Optimize a model with 51 rows, 43 columns and 111 nonzeros
Model has 50 general constraints
Variable types: 13 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 9e+02]
  RHS range        [1e+00, 1e+04]
Presolve removed 31 rows and 10 columns
Presolve time: 0.00s
Presolved: 110 rows, 93 columns, 257 nonzeros
Presolved model has 36 SOS constraint(s)
Variable types: 49 continuous, 44 integer (44 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 5.000000e-01, 47 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.50000    0    9   -0.00000    0.50000      -     -    0s
H    0     0                       0.2000000    0.50000   150%     -    0s
     0     2    0.50000    0    9    0.20000    0.50000   150%     -    0